<a href="https://colab.research.google.com/github/CiceroDEugenio/Advogado_Virtual/blob/main/script_advogado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando pacotes

In [1]:
!pip install -U -q google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('API_KEY_CICERO')

genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
documents = [
    {
        "Titulo": "Pedido de Aposentadoria por Idade - Petição Administrativa",
        "Conteúdo": "Excelentíssimo Senhor Presidente do Instituto Nacional do Seguro Social (INSS), \n\nEu, [SEU NOME], inscrito no CPF [SEU CPF], venho, respeitosamente, por meio desta petição, requerer a concessão do benefício de aposentadoria por idade, conforme previsão legal estabelecida pela Lei [NÚMERO DA LEI], de [DATA DA LEI], e demais dispositivos aplicáveis.\n\n[DESCRIÇÃO DOS FATOS E FUNDAMENTOS JURÍDICOS].\n\nDiante do exposto, requer a Vossa Senhoria a concessão do benefício de aposentadoria por idade, com base nos fundamentos apresentados. \n\nTermos em que,\n\n[LOCAL E DATA].\n\n[ASSINATURA DO REQUERENTE]."
    },
    {
        "Titulo": "Pedido de Aposentadoria por Invalidez - Petição Judicial",
        "Conteúdo": "Excelentíssimo Senhor Juiz de Direito da [VARA JUDICIAL], \n\nEu, [SEU NOME], nacionalidade [NACIONALIDADE], estado civil [ESTADO CIVIL], profissão [PROFISSÃO], inscrito no CPF [SEU CPF], residente e domiciliado na [SEU ENDEREÇO], por intermédio de meu advogado, conforme procuração anexa, vem, respeitosamente, perante Vossa Excelência, propor a presente \n\nAÇÃO PREVIDENCIÁRIA DE CONCESSÃO DE APOSENTADORIA POR INVALIDEZ \n\n[DESCRIÇÃO DOS FATOS E FUNDAMENTOS JURÍDICOS].\n\nDiante do exposto, requer a Vossa Excelência que seja julgada procedente a presente demanda, concedendo o benefício de aposentadoria por invalidez em favor do requerente. \n\nTermos em que,\n\n[LOCAL E DATA].\n\n[ASSINATURA DO REQUERENTE]\n\n[ASSINATURA DO ADVOGADO]"
    },
    {
        "Titulo": "Pedido de Aposentadoria por Tempo de Contribuição - Petição Administrativa",
        "Conteúdo": "Senhor(a) Presidente do Instituto Nacional do Seguro Social (INSS), \n\nEu, [SEU NOME], portador(a) do CPF [SEU CPF], residente e domiciliado(a) na [SEU ENDEREÇO], venho, por meio deste, requerer a concessão do benefício de aposentadoria por tempo de contribuição.\n\n[DESCRIÇÃO DOS FATOS E FUNDAMENTOS JURÍDICOS].\n\nDiante do exposto, requeiro a concessão do benefício de aposentadoria por tempo de contribuição, nos termos da legislação previdenciária vigente. \n\nTermos em que,\n\n[LOCAL E DATA].\n\n[ASSINATURA DO REQUERENTE]."
    },
    {
        "Titulo": "Pedido de Aposentadoria por Idade - Petição Judicial",
        "Conteúdo": "Excelentíssimo Senhor Juiz de Direito da [VARA JUDICIAL], \n\nEu, [SEU NOME], inscrito no CPF [SEU CPF], residente e domiciliado na [SEU ENDEREÇO], venho, por meio de meu advogado, com procuração em anexo, propor a presente \n\nAÇÃO PREVIDENCIÁRIA DE CONCESSÃO DE APOSENTADORIA POR IDADE \n\n[DESCRIÇÃO DOS FATOS E FUNDAMENTOS JURÍDICOS].\n\nDiante do exposto, requer a Vossa Excelência a concessão do benefício de aposentadoria por idade ao requerente. \n\nTermos em que,\n\n[LOCAL E DATA].\n\n[ASSINATURA DO REQUERENTE]\n\n[ASSINATURA DO ADVOGADO]"
    },
    {
        "Titulo": "Pedido de Benefício de Prestação Continuada (LOAS) - Petição Administrativa",
        "Conteúdo": "Excelentíssimo Senhor Presidente do Instituto Nacional do Seguro Social (INSS), \n\nEu, [SEU NOME], inscrito no CPF [SEU CPF], residente e domiciliado na [SEU ENDEREÇO], venho, por meio desta, requerer a concessão do Benefício de Prestação Continuada (LOAS).\n\n[DESCRIÇÃO DOS FATOS E FUNDAMENTOS JURÍDICOS].\n\nDiante do exposto, requer a concessão do Benefício de Prestação Continuada (LOAS), conforme previsto na Lei Orgânica da Assistência Social, em favor do requerente. \n\nTermos em que,\n\n[LOCAL E DATA].\n\n[ASSINATURA DO REQUERENTE]."
    }
]

In [6]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]

In [7]:
model = "models/embedding-001"

In [10]:
def criar_embedding(titulo, conteudo):
  return genai.embed_content(model=model,
                             content=conteudo,
                             title=titulo,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: criar_embedding(row["Titulo"], row["Conteudo"]), axis=1)
df

In [17]:
def executar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]

In [21]:
input_consulta = input("Qual o caso do seu cliente?")
peca_sugerida = executar_consulta(input_consulta, df, model)
print(peca_sugerida)
input_nome = input("Nome do cliente:")
input_data_nascimento = input("Data Nascimento:")
input_cpf = input("CPF:")
input_rg = input("RG:")

prompt = f"Com base no modelo sugerido para a consulta '{peca_sugerida.Conteudo}', crie uma petição inicial com os dados informados pelo usuário:nome {input_nome},data nascimento {input_data_nascimento},cpf {input_cpf},rg {input_rg}"
model_2 = genai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)


Qual o caso do seu cliente?homem 69 anos rural 10 anos de carteira
Titulo        Pedido de Aposentadoria por Idade - Petição Ju...
Conteudo      Excelentíssimo Senhor Juiz de Direito da [VARA...
Embeddings    [-0.0058736824, -0.035543468, -0.054281358, -0...
Name: 3, dtype: object
Nome do cliente:cicero
Data Nascimento:01/01/1965
CPF:12345678901
RG:123456789
Excelentíssimo Senhor Juiz de Direito da [VARA JUDICIAL],

Eu, **Cícero**, inscrito no CPF nº 123.456.789-01, RG nº 12.345.678-9, residente e domiciliado na [SEU ENDEREÇO], venho, por meio de meu advogado, com procuração em anexo, propor a presente

**AÇÃO PREVIDENCIÁRIA DE CONCESSÃO DE APOSENTADORIA POR IDADE**

**I - DOS FATOS E FUNDAMENTOS JURÍDICOS**

Nascido em 01/01/1965, o requerente completou 65 (sessenta e cinco) anos de idade em 01/01/2023, conforme certidão de nascimento anexa, preenchendo assim os requisitos para requerer aposentadoria por idade.

O requerente é segurado do INSS desde [DATA DE INÍCIO DA CONTRIBUIÇÃO], c